# Projektaufgabe - Roundtrip-Path

## Aufgabe: Implementieren Sie einen Roundtrip-Path Planer



1. Gegeben sind  
    1. Startposition  
    2. Mehrere Endpositionen
    3. Das Interface des Roundtrip-Path-Planners soll sich *nicht* von den anderen Bahnplanern unterscheiden.
    4. Der zu verwendete Bahnplanungsalgorithmus soll wählbar sein und geeignet übertragen werden können.
    5. Ausgabe: Ein kollisionsfreier Pfad, der von der gegebenen Startposition alle Endpositionen genau einmal erreicht und wieder zur Startposition zurückführt. Dabei soll der Gesamtpfad möglichst kurz sein. Koordinieren Sie den Pfad geeignet (z.B. farblich), so dass sich erkennen lässt, was Start-Punkt, kollisionsfreier Zwischenpfad und Zielpunkte sind. \
    *Anmerkung: NetworkX hat prinzipiell die Möglichkeit auf einem gegebenen Graph dieses Problem zu lösen, sogar auf verschiedene Arten und Weisen. Sie können dies nutzen*

## Roundtrip-Path Planner - single-query, naiver Ansatz

<img src="img/single_query_RP.png" width="1500"/>

In [ ]:
import networkx as nx
from IPPerfMonitor import IPPerfMonitor

import math
from typing import List, Any
import numpy as np
from numpy.typing import NDArray

import matplotlib.pyplot as plt
import traceback
from IPBenchmark import Benchmark
from shapely.geometry import Point, Polygon, LineString
from typing import Tuple, Type, Callable
from networkx.classes.reportviews import NodeView


import IPTestSuite as ts
from IPEnvironment import CollisionChecker
from IPVISBasicPRM import basicPRMVisualize
from IPVISLazyPRM import lazyPRMVisualize
from IPVISVisibilityPRM import visibilityPRMVisualize
from IPVISLazyPRM import lazyPRMVisualize
from IPVISVisibilityPRM import visibilityPRMVisualize
from IPVisibilityPRM import VisPRM
from IPBasicPRM import BasicPRM
from IPLazyPRM import LazyPRM

from IPEnvironmentKin import animateSolution
from IPEnvironment import animatePointRobotSolution
import IPLazyPRM
import IPVISLazyPRM
from IPPlanarManipulator import PlanarJoint, PlanarRobot
from IPEnvironmentKin import planarRobotVisualize
from IPEnvironmentKin import KinChainCollisionChecker

In [ ]:
class Roundtrip_path_planner:
    """
    A class to plan a roundtrip path that visits all start and goal nodes.
    """


    def __init__(self, path_planner, collision_checker: CollisionChecker):
        self.plannerClass = path_planner # Class of the path planner
        self._collisionChecker = collision_checker
        self.graph = nx.Graph() # graph to store the Traveling Salesman Graph (TSG) containing only start and goal nodes. Each edge contains the actual path between the nodes and the respective summed weight.
        self.planners = [] # List of instantiated path planners
        self.paths = [] # List of paths computed by the path planners

    
    def _update_graph_edge_weights_vectorized(self):
        pos = nx.get_node_attributes(self.graph, 'pos')
        edge_array = np.array([
            np.linalg.norm(np.array(pos[u]) - np.array(pos[v]))
            for u, v in self.graph.edges()
        ])
        
        for (u, v), w in zip(self.graph.edges(), edge_array):
            self.graph[u][v]['distance'] = w

    def _compute_path_length(self, graph: nx.Graph, path: List[Any]) -> float:
        """
        Computes the total length of a path in the graph.
        Args:
            graph (nx.Graph): The graph containing the path.
            path (List[Any]): A list of nodes representing the path.
        returns:
            float: The total distance of the path. Returns 0.0 if the path is empty or has less than 2 nodes.
        """
        if not path or len(path) < 2:
            return 0.0
        total_distance = sum(
            math.dist(graph.nodes[u]['pos'], graph.nodes[v]['pos'])
            for u, v in zip(path[:-1], path[1:])
        )
        return total_distance
    

    @IPPerfMonitor
    def planPath(self, startList: List[List[int|float]], goalList: List[int|float], config) -> List[str]:
        """
        Plans a roundtrip path that visits all start and goal nodes.
        Args:
            startList (array): start position in planning space. E.g. [[1,2]]
            goalList (array) : goal position in planning space. E.g. [[3,4]]
            config (dict): dictionary with the needed information about the configuration options

        Returns:
            List[List[Any]]: A list representing the roundtrip path visiting all goals and returning to the start.
        """
        # add start and target nodes to the graph
        for i, coordinates in enumerate(startList):
            self.graph.add_node("start", pos=coordinates)
        for i, coordinates in enumerate(goalList):
            self.graph.add_node(f"goal_{i}", pos=coordinates)
        # get added target nodes
        target_nodes = self.graph.nodes
        # Copy the TSG graph to the main graph
        self.graph = self.graph.copy()
        
        
        # for each target nodes pair compute the path between them
        for i, start_node in enumerate(target_nodes):
            start_node = str(start_node)  # Convert to string for consistency
            for j, goal_node in enumerate(target_nodes):
                if j <= i:
                    continue
                plannerInstance = self.plannerClass(self._collisionChecker)
                goal_node = str(goal_node) # Convert to string for consistency
                # plan path from start_node to goal_node
                path: List[Any] = plannerInstance.planPath([self.graph.nodes[start_node]['pos']], [self.graph.nodes[goal_node]['pos']], config)

                self.paths.append(path)  # Store the path for later visualization
                self.planners.append(plannerInstance)  # Store the planner instance for later visualization
                
                # Relabel nodes so composition does not merge different nodes (different pos)
                # TODO: check if relabeling can be avoided
                mapping = {node: (start_node if node == "start" else goal_node if node == "goal" else f'{start_node}_{goal_node}_{node}') for node in plannerInstance.graph.nodes}
                
                # Create a subgraph containing only the nodes and edges in the planned path
                path_subgraph = plannerInstance.graph.subgraph(path).copy()
                # Relabel the nodes in the path subgraph
                path_subgraph = nx.relabel_nodes(path_subgraph, mapping)
                # Merge the path subgraph into the main graph
                self.graph = nx.compose(self.graph, path_subgraph)

        # compute distances
        self._update_graph_edge_weights_vectorized()

        # Use the Traveling Salesman Problem approximation to find a roundtrip path
        try:
            roundtrip_path = np.asarray(nx.algorithms.approximation.traveling_salesman_problem(
                self.graph,
                weight='distance',
                nodes=target_nodes,
                cycle=True
            ))
            return roundtrip_path.tolist()
        except:
            return []




2. Evaluieren Sie ihr Konzept mit BasicPRM, LazyPRM, VisibilityPRM anhand von mindestens 6 Benchmarkumgebungen (3 * 2-DoF Punktroboter, 3 * Planarroboter). Die Szenen der Benchmarkaufgaben für die 2-DoF Punktroboter sollen unterschiedlich aussehen und unterschiedlich schwer sein. Die Szene der Benchmarkaufgaben für die Planarroboter können gleich sein und lediglich die Anzahl der Freiheitsgrade und Start, Ziel und Zwischenpunkte verändert werden.
    1. Betrachten Sie z.B. Anzahl der Kollisionsberechnungen, Planungszeit, Roadmapgröße, Länge Lösungspfad und Stellen Sie die Ergebnisse graphisch dar und diskutieren Sie diese.
    2. Für die 2-DoF Punkt- und Planarroboter stellen Sie sowohl die Vorgehensweise des Roundtrip-Planers (welche Bahnen werden wann geplant, was ist das finale Ergebnis) als auch das Endergebnis als Animation (s. IP-10-0-PlanarManipulator.ipnyb) dar.

In [ ]:
# Factory for planners
plannerFactory: dict[str, List[Any]] = dict() # plannerFactory[plannerName] = [plannerClass, configDict, visualizerFunction]


# plannerFactory["basePRM"] = [
#     BasicPRM, 
#     {
#         "radius": 3, 
#         "numNodes": 300,
#         "useKDTree": True
#     }, 
#     basicPRMVisualize]


plannerFactory["lazyPRM"] = [
    LazyPRM,
    {
        "initialRoadmapSize": 40, 
        "updateRoadmapSize": 20, 
        "kNearest": 5,
        "maxIterations": 40
    },
    lazyPRMVisualize]


plannerFactory["visibilityPRM"] = [
    VisPRM,
    {
        "ntry": 40,
        "reset_graph": True
    },
    visibilityPRMVisualize]


In [ ]:
class ResultCollection (object):
    
    def __init__(self, plannerFactoryName, roadMapPlanner, benchmark: Benchmark, solution, perfDataFrame):
        self.plannerFactoryName = plannerFactoryName    # e.g. "basePRM", "lazyPRM", "visibilityPRM"
        self.roadMapPlanner = roadMapPlanner            # the roundtrip path planner instance used to compute the roundtrip path
        self.benchmark = benchmark                      # the benchmark used to compute the path
        self.solution = solution                        # the computed path as a list of nodes
        self.perfDataFrame = perfDataFrame              # performance data collected during the planning process

In [ ]:
def RoadMapPlannerVisualize(
        roundtrip_planner,
        benchmark: Benchmark, 
        solution: List[Any],
        planner_visualizer: Callable,
        multiquery=False) -> None:
    """
    Visualizes the roundtrip path planner and the solution path.
    Args:
        roundtrip_planner: The roundtrip path planner instance.
        benchmark: The benchmark used.
        solution (List[Any]): The computed path as a list of nodes.
        planner_visualizer (Callable): The visualizer function for the path planner.
    """


    fig_local = plt.figure(figsize=(10,10))
    ax = fig_local.add_subplot(1,1,1)
    title = roundtrip_planner.plannerClass.__name__
    title += " " + benchmark.name
    if solution == []:
        title += " (No path found!)"
    title += "\n Assumed complexity level " + str(benchmark.level)
    ax.set_title(title)



    # Plot the roadmap planner.graph
    pos = nx.get_node_attributes(roundtrip_planner.graph, 'pos')

    if multiquery:
        planner_visualizer(roundtrip_planner.plannerInstance, solution)
    else:
        basicPRMVisualize(roundtrip_planner, solution)

    # get start and goal nodes by position. Therefore this function is independent of the nodes labeling convention.
    start_nodes = [
        node for node, data in roundtrip_planner.graph.nodes(data=True)
        if 'pos' in data and data['pos'] in benchmark.startList
    ]
    goal_nodes = [
        node for node, data in roundtrip_planner.graph.nodes(data=True)
        if 'pos' in data and data['pos'] in benchmark.goalList
    ]
    #target_nodes = [str(i) for i in range(len(target_positions))]  # use string node names like "0", "1", ...
    nx.draw_networkx_nodes(roundtrip_planner.graph, 
                            pos,
                            nodelist=start_nodes + goal_nodes,
                            node_size=300,
                            node_color='#00dd00', 
                            ax = ax)
    
    
    
    labels = {node: "S" if node in start_nodes else "G" for node in goal_nodes}
    nx.draw_networkx_labels(roundtrip_planner.graph, pos, labels=labels, ax=ax)
    
    if not multiquery:
        # Plot each planner's graph in a separate subplot
        num_planners = len(roundtrip_planner.planners)
        if num_planners > 0:
            fig_subplots, axes = plt.subplots(1, num_planners, figsize=(5*num_planners, 5))
            if num_planners == 1:
                axes = [axes]
            for i, (path_planner, ax_sub) in enumerate(zip(roundtrip_planner.planners, axes)):
                ax_sub.set_title(f"Subpath {i+1}")
                # Clear the axis before plotting to avoid overlapping drawings (especially obstacles)
                #ax_sub.clear()
                # Optionally, set limits or background here if needed
                planner_visualizer(path_planner, roundtrip_planner.paths[i], ax=ax_sub, nodeSize=50)
                plt.tight_layout()

"""
                axes = [axes]
            for i, (path_planner, ax_sub) in enumerate(zip(roundtrip_planner.planners, axes)):
                ax_sub.set_title(f"Subpath {i+1}")
                planner_visualizer(path_planner, roundtrip_planner.paths[i], ax=ax_sub, nodeSize=50)
            plt.tight_layout()

"""



In [ ]:
def run_benchmarks(roadmapPlannerClass, plannerFactory) -> List[ResultCollection]:
    """
    Runs all planners on the benchmarks and collects the results.
    Args:
        roadmapPlannerClass: The class of the roundtrip path planner to be used.
        plannerFactory (dict): A dictionary containing the planner factory, where each key is a planner name and the value is a list containing the planner class, configuration dictionary, and visualizer function.
    Returns:
        List[ResultCollection]: A list of ResultCollection objects containing the results of the path planning.
    """

    resultList: List[ResultCollection] = list()
    testList: List[Any] = ts.benchList

    for key, producer in list(plannerFactory.items()):
        print(key, producer)
        for benchmark in testList:
            plannerClass, config, visualizerFunction = producer
            print ("Planning: " + key + " - " + benchmark.name)
            #planner = plannerClass(benchmark.collisionChecker)
            roadmapPlanner = roadmapPlannerClass(plannerClass, collision_checker=benchmark.collisionChecker,)
            IPPerfMonitor.clearData()
            #try:
            resultList.append(ResultCollection(
                plannerFactoryName=key,
                roadMapPlanner=roadmapPlanner,
                benchmark=benchmark,
                solution=roadmapPlanner.planPath(
                    startList=benchmark.startList,
                    goalList=benchmark.goalList,
                    config=config
                ),
                perfDataFrame=IPPerfMonitor.dataFrame()
            ))
            #except Exception as e:
            #    print ("PLANNING ERROR ! PLANNING ERROR ! PLANNING ERROR ", e)
            #    pass
    return resultList


def plot_results(resultList: List[ResultCollection], plannerFactory, multiquery=False) -> None:
    """
    Plots the results of the path planning.
    Args:
        resultList (List[ResultCollection]): A list of ResultCollection objects containing the results of the path planning.    
    """
    for result in resultList:
        #try:
        RoadMapPlannerVisualize(
            roundtrip_planner= result.roadMapPlanner,
            benchmark= result.benchmark, 
            solution= result.solution,
            planner_visualizer= plannerFactory[result.plannerFactoryName][2],
            multiquery=multiquery)

In [ ]:
# Run the benchmarks and collect the results
resultList = run_benchmarks(Roundtrip_path_planner, plannerFactory)

In [ ]:
resultList[0].solution

In [ ]:
# Plot the results
plot_results(resultList, plannerFactory, multiquery=False)

In [ ]:
# Animations for the 2-DoF Robot benchmark solutions
def animatePlanarRobotResults(resultList, PRMVisualizerFunction=basicPRMVisualize, step=0.5):
    for result in resultList:
        checker = result.benchmark.collisionChecker
        if isinstance(checker, KinChainCollisionChecker):
            animateSolution(result.roadMapPlanner, 
                        checker, 
                        result.benchmark,
                        result.solution, 
                        PRMVisualizerFunction, step) # always use basicPRMVisualize, as this works also for out single queue roundtrip planners

In [ ]:
# Animations for the 2-DoF Point Robot benchmark solutions
def animatePointRobotResults(resultList, PRMVisualizerFunction=basicPRMVisualize, step=0.5):
    for result in resultList:
        checker = result.benchmark.collisionChecker
        if not isinstance(checker, KinChainCollisionChecker):
            animatePointRobotSolution(result.roadMapPlanner, 
                        checker, 
                        result.benchmark,
                        result.solution, 
                        PRMVisualizerFunction, step) # always use basicPRMVisualize, as this works also for out single queue roundtrip planners

In [ ]:
animatePointRobotResults(resultList)

In [ ]:
animatePlanarRobotResults(resultList)

3. In einem weiteren Schritt entwickeln Sie eine spezielle Variante auf Basis des VisibilityPRM. Wie Sie wissen, ist dieser für Multi-Query Anfragen vorteilhaft nutzbar. Nutzen Sie dies für das Roundtrip-Konzept. Hier müssen Sie eventuell die Art wie Sie den Visibility-PRM aufrufen geschickt verändern. Führen Sie ebenfalls die Evaluation an den Benchmarkumgebungen durch und vergleichen Sie die Ergebnisse mit den Versionen aus 2.) 

## Roundtrip-Path Planner - multi-query using VisPRM

<img src="img/multi_query_RP.png" width="1500"/>

In [ ]:
class Multi_query_roundtrip_path_planner:

    def __init__(self, path_planner: Type[Any], collision_checker: CollisionChecker):
        assert hasattr(path_planner, "planPath"), "path_planner must have a method called 'planPath'"
        self.graph = nx.Graph() # graph to store all paths between start and goal nodes
        self.plannerClass = path_planner
        self._collisionChecker = collision_checker
        self.plannerInstance = self.plannerClass(self._collisionChecker)

    def _update_graph_edge_weights_vectorized(self):
        pos = nx.get_node_attributes(self.plannerInstance.graph, 'pos')
        edge_array = np.array([
            np.linalg.norm(np.array(pos[u]) - np.array(pos[v]))
            for u, v in self.plannerInstance.graph.edges()
        ])

        for (u, v), w in zip(self.plannerInstance.graph.edges(), edge_array):
            self.plannerInstance.graph[u][v]['distance'] = w


    @IPPerfMonitor
    def planPath(self, startList: List[List[Any]], goalList: List[List[Any]], config) -> List[Any]:
        """
        Plans a roundtrip path that visits all start and goal nodes.
        Args:
            startList (array): start position in planning space. E.g. [[1,2]]
            goalList (array) : goal position in planning space. E.g. [[3,4]]
            config (dict): dictionary with the needed information about the configuration options

        Returns:
            List[List[Any]]: A list representing the roundtrip path visiting all goals and returning to the start.
        """
        #target_nodes: nx.classes.reportviews.NodeView = self.plannerInstance.graph.nodes
        target_nodes = []

        # Compute path from start to each other target node.
        for i, goal_coordinates in enumerate(goalList):
            # plan path from start_node to goal_node
            path = self.plannerInstance.planPath([startList[0]], [goal_coordinates], config)

            # If no path from start to the target is found, return an empty list.
            if len(path) < 2:
                # remove the self-loop edge if it exists
                if self.plannerInstance.graph.has_edge("start", "start"):
                    self.plannerInstance.graph.remove_edge("start", "start")
                return []

            self.plannerInstance.graph = nx.relabel_nodes(
                self.plannerInstance.graph,
                {"goal": f"goal_{i}"},
                copy=False
            )
            target_nodes.append(f"goal_{i}")  # Collect target nodes for TSP solver


        # Convert names to strings to ensure consistency with the TSP solver
        self.plannerInstance.graph = nx.relabel_nodes(
            self.plannerInstance.graph,
            {node: str(node) for node in self.plannerInstance.graph.nodes},
            copy=False
        )

        # remove the self-loop edge if it exists
        if self.plannerInstance.graph.has_edge("start", "start"):
            self.plannerInstance.graph.remove_edge("start", "start")

        # Add edge weights to the graph
        self._update_graph_edge_weights_vectorized()


        # Use the Traveling Salesman Problem approximation to find a roundtrip path
        # Compared to the single queue approach, we apply the TSP solver to the graph itself.
        tsg_solution = np.asarray(nx.algorithms.approximation.traveling_salesman_problem(
            self.plannerInstance.graph,
            weight='distance',
            nodes=["start"] + target_nodes,  # Include start node in the TSP solver
            cycle=True
        ))



        if len(tsg_solution) < 2:
            return []
        
        self.graph = self.plannerInstance.graph

        return list(tsg_solution)


In [ ]:
# Factory for multi-query planners
multiqueuePlannerFactory: dict[str, List[Any]] = dict() # each element contains plannerClass, configDict and visualizerFunction

multiqueuePlannerFactory["visibilityPRM"] = [
    VisPRM,
    {
        "ntry": 40,
        "reset_graph": False
    },
    visibilityPRMVisualize]

In [ ]:
# Run the benchmarks and collect the results
multi_query_result_list = run_benchmarks(Multi_query_roundtrip_path_planner, multiqueuePlannerFactory)

In [ ]:
# Plot the results for all Roadmap Planners showing path length, planning time and roadmap size
# TODO: for naive approach: sum all roadmap sizes or only from final roadmap?

import IPTestSuite as ts

testList = ts.benchList
for bench in testList:
    title = bench.name
    pathLength = dict()
    planningTime = dict()
    roadmapSize  = dict()
    nCollisions = dict()
    
    for result in resultList + multi_query_result_list:
        if result.benchmark.name == bench.name:
            label = f"{type(result.roadMapPlanner).__name__} \n {result.plannerFactoryName}"
            #print result.benchmark.name  + " - " +  result.plannerFactoryName, len(result.solution)
            pathLength[label] = len(result.solution)
            planningTime[label] = result.perfDataFrame.groupby(["name"]).sum(numeric_only=True)["time"]["planPath"]
            nCollisions[label] = (result.perfDataFrame['name'] == 'pointInCollision').sum()

            if type(result.roadMapPlanner) == Multi_query_roundtrip_path_planner:
                roadmapSize[label] = result.roadMapPlanner.graph.size()
            else:
                roadmapSize[label] = 0
                for planner in result.roadMapPlanner.planners:
                    roadmapSize[label] += planner.graph.size()


            

    fig, ax = plt.subplots()
    plt.xticks(rotation=45)
    width = 0.2

    ax.bar(np.arange(len(pathLength.keys())), pathLength.values(),width, color="blue")
    ax.set_ylabel(title + " Number of nodes in path", color="blue")
    ax.set_xticks(np.arange(len(pathLength.keys())) + width)
    ax.set_xticklabels(pathLength.keys())

    ax2 = ax.twinx()
    bar = ax2.bar(np.arange(len(pathLength.keys()))+width, planningTime.values(),width, color="red")
    ax2.set_ylabel(title + " Planning time", color="y")

    # Add coloring and patterns on axis two
    hatches = ['x' if length==0 else '' for length in pathLength.values()]
    color   = ['red' if length==0 else 'yellow' for length in pathLength.values()]
    for i,thisbar in enumerate(bar.patches):
        thisbar.set_facecolor(color[i])
        thisbar.set_hatch(hatches[i])

    # Multiple axes 
    ax3 = ax.twinx()
    ax3.bar(np.arange(len(pathLength.keys()))+2*width, roadmapSize.values(),width, color="purple")
    ax3.set_ylabel(title + " Roadmap size",  color="purple")
    ax3.spines['right'].set_position(('axes', 1.15))
    ax3.spines['right'].set_color("purple")

    # Collision Checks
    ax4 = ax.twinx()
    ax4.bar(np.arange(len(pathLength.keys()))+3*width, nCollisions.values(), width, color="green")
    ax4.set_ylabel(title + " Number of collision checks", color="green")
    ax4.spines['right'].set_position(('axes', 1.25))
    ax4.spines['right'].set_color("green")


    
    

# Statistiken

# Punkt Roboter

# Plus Sign
## Roundtrip basePRM
- Schlechter Performance aber Parameterabhängig
- Kein resampling vorgesehen

## Roundtrip lazyPRM
- Weniger initial Nodes gewählt da resampling möglich
- Weniger Kollisionsabfragen daher geringere Planning time

## Roundtrip visibilityPRM
- Geringste Anzahl Nodes der single Query Algorithmen aufgrund dessen Konfigurationsauswahl
- Höhere Laufzeit als Lazy PRM aufgrund der Kollisionsabfragen in Roadmap Erstellung
- Verwerfen vieler Nodes

## Roundtrip visibilityPRM multi query
- Ähnlich Performanz wie Lazy PRM bei geringer Anzahl von Zielpunkten
- Kommutierte Roadmapsize kleiner da keine mehrfache Überdeckung

# Square Grind
## Roundtrip visibilityPRM
- Hohe Laufzeit, da viele Nodes nötig für hohe Abdeckung


# Harder Square Grid
- Performanz generell schlechter aufgrund höherer Anzahl an Hindernissen

# 2 DoF Planarroboter

# 1 Hindernis
## Roundtrip basePRM
- Geringe Performanz durch Kollisionsabfragen
## Roundtrip lazyPRM
- Sehr gute Performanz durch geringe Anzahl teurer Kollisionsabfragen
- Häufiges Resampling
## Roundtrip visibilityPRM
- Nur ein Start und Zielpunkt daher gleich zu multi query

# 2 Hindernisse
## Roundtrip visibilityPRM multi query
- Schnellere multi query da mehr Zielpunkte

# 4 Hindernisse
## Roundtrip basePRM
## Roundtrip lazyPRM
## Roundtrip visibilityPRM
## Roundtrip visibilityPRM multi query



In [ ]:
multi_query_result_list[0].perfDataFrame

In [ ]:
df = multi_query_result_list[0].perfDataFrame
n_rows = df.shape[0]
n_rows

In [ ]:
# Plotting the results
plot_results(multi_query_result_list, plannerFactory=multiqueuePlannerFactory, multiquery=True)

In [ ]:
animatePlanarRobotResults(multi_query_result_list)
#animatePointRobotResults(multi_query_result_list, PRMVisualizerFunction=visibilityPRMVisualize)

## Wie können Sie die Bewegungsbahnen optimieren/Glätten? Erläutern Sie kurz eine mögliche Vorgehensweise.

<p align="center">
    <img src="img/observed_shortcut_possibility.png"  width="600"/>
</p>

### Beobachtung: 
- Wenn Ziel mit "Connection" verbunden, kann häufig direkt mit einem guard verbunden werden
- Außerdem kann Weg mit weiteren Shortcuts verkürzt 

### Möglicher Ansatz: Deterministic Shortcuts & Inverse Rounding

#### Deterministic Shortcuts
Sortiere die Punkte entlang des Weges. Je kleiner der Winkel, desto höher die Priorität:



$$
\huge
Q_{p_2} = \frac{|\overline{p_1p_2}| + |\overline{p_2p_3}|}{|\overline{p_1p_3}|}
$$

<p align="center">
    <img src="img/priority_sorted.png"  width="600"/>
</p>

Für jeden Punkt:
- Überprüfe ob shortcut zwischen den beiden Nachbarpunkten im Weg möglich ist.
- Falls nicht: Operation DelTree

<p align="center">
    <img src="img/delTree.png"  width="600"/>
</p>



### Inverse Rounding
- Ziel: Keine plötzlichen Richtungsänderungen, sondern sanftere Kurven
- Nach Deterministic Shortcuts: Weg verläuft eng an den Hindernissen

<p align="center">
    <img src="img/inverse_rounding_3.png"  width="600"/>
</p>

Nach der Optimierung mittels der Shortcut Strategy, verläuft der Weg eng an Hindernissen entlang, wenn er die Richtung wechselt. Daher eignet sich hier der "Inverse Rounding" Ansatz. Hierbei werden Kurven erzeugt, die den Weg um ein Objekt nicht enger machen (siehe Abbildung 1). Dies lässt sich z.B. modellieren durch:


$$
\huge
\overrightarrow{z(t)} = r \cdot \overrightarrow{P_{2n} P_1} (t-1)^2 + r \cdot \frac{|\overrightarrow{P_{2n} P_1}|}{|\overrightarrow{P_{2n} P_3}|} \cdot \overrightarrow{P_{2n} P_3} t^2 + P_{2n}
$$

mit 

$$
\huge
\overrightarrow{z(0,5)} = P_2
$$

wobei r der rounding factor ist, $\overrightarrow{z(t)}$ die Wegposition zum Zeitpunkt $t$ darstellt und $P_1$, $P_2$ und $P_3$ benachbarte Punkte sind.

Erkläutern Sie bitte zudem im Endbericht (mind. 1 Seite):
1. Wie funktioniert der Roundtrip-Planer, den Sie verwenden im Detail und warum haben Sie diesen gewählt.
2. Wie können Sie die Bewegungsbahnen optimieren/Glätten? Erläutern SIe kurz eine mögliche Vorgehensweise.

Anmerkung: Bitte checken Sie das Notebook "IP-X-0-Benchmarking-concept.ipnyb" und "IP-X-1-Automated_PlanerTest.ipnyb" für Profiling und Statistiken.

## Endbericht

## Wie funktioniert der Roundtrip-Planer, den Sie verwenden im Detail und warum haben Sie diesen gewählt.

### Single-queue Roundtrip Planner
Wir haben für dieses Projekt zwei Roundtrip-Planner erstellt und verglichen. Zunächst haben wir einen Roundtrip-Planner erstellt, welcher flexibel beliebige planner nutzen kann und single-queue Anfragen an diese Stellt. Gegeben n start oder Zielpunkte, berechnet dieser Roundtrip-Planner alle n! Wege zwischen den jeweiligen Punktepaaren. Für jeden der Paare wird ein planner instanziiert. Die jeweiligen gefundenen Wege werden in einen gemeinsamen Graphen zusammengeführt. Dieser Graph wird verwendet um mittels des der networkX Bibliothek das "traveling salesman" Problem zu lösen.

Diesen Ansatz haben wir als naiven Ansatz gewählt um ihn mit dem multi-query Ansatz zu vergleichen. Der naive Ansatz hat zum Vorteil, dass beliebige Planner genutzt werden können. Für die Implementierung mussten die aus der Vorlesung vorgegebenen planner nicht angepasst werden. Dagegen hat dieser Ansatz zum Nachteil, dass er die Roadmaps der planner nicht wiederverwendet, sondern ausschließlich den gefundenen Weg verwendet. Außerdem müssen für n Zielpunkte (einschließlich des Start Punktes) n! Wege berechnet werden. Damit skaliert dieser Ansatz sehr schlecht für eine hohe Anzahl von Zielpunkten.

Für dieses Projekt haben wir uns dazu entschieden diesen Ansatz dennoch zu untersuchen, um einen naiven Ansatz zum Vergleich zu dem multi-queue Ansatz zu haben. Weitere Optimierungen wären jedoch umsetzbar, welche die Effizienz erhöhen würden. Die planner könnten zum Beispiel die roadmap des vorherigen planners übernehmen und diese Erweitern. Hierfür hätte man aber die planner anpassen müssen. Außerdem stellt die Anforderung an die planner Klasse, dass durch die Übernahme des Graphen keine Bedingungen des Algorithmus (wie Prioritäten) verletzt werden.

Durch Anwendung von Heuristiken könnte zudem der Algorithmus beschleunigt werden, indem nicht alle n! Wege berechnet werden. Da wir den Algorithmus unabhängig von Annahmen zu Heuristiken machen wollten, haben wir diese Strategie hier nicht verfolgt.

### Multi-queue Roundtrip Planner

Für den Multi-queue Roadmap Planner haben wir den VisibilityPRM planner genutzt.der Vorteil des VisibilityPRM planers ist, dass er sich für multi-queue Anfragen eignet. Wir haben den Code für den planer so angepasst, dass nur einmal eine roadmap erstellt wird und mehrere Male ein Weg angefragt werden kann.

Der Roadmap Planner erzeugt zunächst mittels VisibilityPRM eine Roadmap, doe für alle Anfragen genutzt wird. Anschließend werden alle Wege von dem start Punkt aus zu allen anderen Zielpunkten gesucht. Somit werden für n Zielpunkte n-1 Anfragen gestellt. Existiert einer der Wege nicht, terminiert der Roundtrip Planner und gibt keine Lösung aus. Die gefundenen Wege werden jeweils mit der Roadmap vereint. Dadurch werden die Zielpunkte mit der Roadmap verbunden.

Anschließend wird der resultierende Graph (die Roadmap mit allen Verbindungen zu den Zielknoten) genutzt um mittels der NetworkX Bibliothek das traveling salesman problem zu lösen.

### Vergleich der beiden Roadmap Planner

Im Vergleich zum naiven Ansatz, welcher $\frac{n(n-1)}{2}$ Anfragen an die planner stellt, stellt der multi-query Ansatz nur $n-1$ Anfragen. Zudem erstellt der naive Ansatz n! Roadmaps, während der multi-query Ansatz nur eine Roadmap erstellt, die er mit den Verbindungen zu den Zielpunkten erweitert.

TODO: VERGLEICH MITTELS STATISTIK

## Wie können Sie die Bewegungsbahnen optimieren/Glätten? Erläutern Sie kurz eine mögliche Vorgehensweise.
Die Bewegungsbahnen können optimiert werden, idem eine shortcut Strategie angewandt wird und anschließend die Ecken geglättet werden. Im folgenden Stellen wir eine Möglichkeit dieses Ansatzes for.

### Shortcut Strategy
Eine verbreitete Methode zur Pfadoptimierung ist das Deterministic Shortcut-Verfahren nach Bechthold & Glavina. Dabei wird der gefundene Pfad iterativ verbessert, indem für zufällig oder systematisch ausgewählte Pfadsegmente geprüft wird, ob eine direkte Verbindung zwischen deren Endpunkten kollisionsfrei möglich ist. Ist dies der Fall, wird das Segment durch die direkte Verbindung ersetzt, wodurch der Pfad kürzer und glatter wird. Dieser Vorgang wird mehrfach wiederholt, bis keine weiteren Abkürzungen möglich sind oder ein Abbruchkriterium erreicht ist. Das Verfahren ist einfach zu implementieren und führt oft zu deutlich kürzeren und glatteren Bewegungsbahnen, ohne die Kollisionsfreiheit zu verletzen.

In der Vorlesung haben wir gesehen, dass dieser deterministische Ansatz tendenziell schneller konvergiert als der behandelte "Pure Probabilistic" der der „Extended“ Probabilistic (Latombe).

### Termination
Die Optimierung bricht ab, wenn sich die Pfadlänge über viele Iterationen kaum noch ändert. Ist die Varianz der letzten Iterationen unter einem bestimmten Schwellenwert v_min, so wird die Optimierung abgebrochen.

```plaintext
Pseudocode:

Daten: Pfad p ist eine sortierte Menge von Stützpunkten
Daten: Schlange q

l = p.LÄNGE();
q.PUSH(l);
wenn q.GRÖSSE() < smax dann
    zurück falsch;
q.POP();  // Ältestes Element entfernen
wenn q.VARIANZ() < vmin dann
    zurück wahr;
zurück falsch;
```

### Glättung
Durch den oben beschriebenen Ansatz wird der Weg verkürzt. Zudem kann der Weg noch geglättet werden, was vermeidet, dass der Roboter an jedem Punkt vollständig anhalten muss. Eine Rundung verlängert zwar die Strecke, kann jedoch je nach Roboter für eine schnellere und kontinuierlicher Ausführung sorgen.

Nach der Optimierung mittels der Shortcut Strategy, verläuft der Weg eng an Hindernissen entlang, wenn er die Richtung wechselt. Daher eignet sich hier der "Inverse Rounding" Ansatz. Hierbei werden Kurven erzeugt, die den Weg um ein Objekt nicht enger machen (siehe Abbildung 1). Dies lässt sich z.B. modellieren durch:


$$
\overrightarrow{z(t)} = r \cdot \overrightarrow{P_{2n} P_1} (t-1)^2 + r \cdot \frac{|\overrightarrow{P_{2n} P_1}|}{|\overrightarrow{P_{2n} P_3}|} \cdot \overrightarrow{P_{2n} P_3} t^2 + P_{2n}
$$

mit 

$$
\overrightarrow{z(0,5)} = P_2
$$

wobei r ein skalierungsfaktor ist, $\overrightarrow{z(0,5)}$ die Wegposition zum Zeitpunkt $t$ darstellt und $P_1$, $P_2$ und $P_3$ benachbarte Punkte sind.

<img src="img/inverse_rounding_1.png" alt="Abbildung 1" width="300"/>
Abbildung 1